In [10]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from datetime import datetime
import os

In [2]:
# Charger les données
data = pd.read_csv('Données/dataset.csv')

# Remplir les valeurs manquantes avec 0 (si nécessaire)
#data.fillna(0, inplace=True)

In [3]:
# Sélectionner les fonctionnalités
features = ['FTHG', 'FTAG', 'HTGS', 'ATGS', 'HTGC'
            , 'ATGC', 'HTP', 'ATP', 'MW', 'HTFormPts', 'ATFormPts'
            , 'HTWinStreak3', 'HTWinStreak5', 'HTLossStreak3', 'HTLossStreak5'
            , 'ATWinStreak3', 'ATWinStreak5', 'ATLossStreak3', 'ATLossStreak5'
            , 'HTGD', 'ATGD', 'DiffPts', 'DiffFormPts']

In [4]:
# Initialiser les modèles de régression linéaire
models = {feature: LinearRegression() for feature in features}
scalers = {feature: StandardScaler() for feature in features}

In [5]:
# Entraîner un modèle pour chaque fonctionnalité
for feature in features:
    X = data[features].drop(columns=[feature]) # Retirer la colonne que l'on souhaite prédire
    y = data[feature]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Normaliser
    X_train_scaled = scalers[feature].fit_transform(X_train)
    X_test_scaled = scalers[feature].transform(X_test)

    models[feature].fit(X_train_scaled, y_train)

    y_pred = models[feature].predict(X_test_scaled)
    mse = np.mean((y_pred - y_test) ** 2)
    print(f"MSE pour {feature}: {mse}")

MSE pour FTHG: 1.4440130382663776
MSE pour FTAG: 1.2180614077561605
MSE pour HTGS: 22.92092061288901
MSE pour ATGS: 23.312658800405337
MSE pour HTGC: 19.52716498801564
MSE pour ATGC: 21.31147703626085
MSE pour HTP: 1.925598256452369e-30
MSE pour ATP: 3.0228044877316518e-30
MSE pour MW: 4.581460978512325
MSE pour HTFormPts: 2.9539711190718214
MSE pour ATFormPts: 2.951828511720623
MSE pour HTWinStreak3: 0.03894659750470024
MSE pour HTWinStreak5: 0.01366547225081605
MSE pour HTLossStreak3: 0.03484590185547092
MSE pour HTLossStreak5: 0.010255215196860437
MSE pour ATWinStreak3: 0.03621295349910149
MSE pour ATWinStreak5: 0.01120446373399995
MSE pour ATLossStreak3: 0.03222532049682414
MSE pour ATLossStreak5: 0.006964885716210765
MSE pour HTGD: 0.06767812570219839
MSE pour ATGD: 0.06508168607975043
MSE pour DiffPts: 2.0737817089464915e-30
MSE pour DiffFormPts: 0.022152394676078425


In [16]:
# Saisir l'équipe à domicile et l'équipe à l'extérieur
home_team = 'Chelsea' #input("Enter the Home Team: ")
away_team = 'Arsenal' #input("Enter the Away Team: ")

In [17]:
# Filtrer les données pour les équipes spécifiques
filtered_data = data.loc[np.where((data['HomeTeam'] == home_team) & (data['AwayTeam'] == away_team))] #data.loc[np.where((data['HomeTeam'] == home_team) | (data['AwayTeam'] == away_team))]

In [18]:
filtered_data.shape

(17, 40)

In [19]:
if not filtered_data.empty:

    # Filtrer uniquement les colonnes numériques pour le calcul de la moyenne
    combined_data_numeric = filtered_data[features].mean(axis=0)

    # Créer une DataFrame temporaire pour contenir les valeurs moyennes des caractéristiques / .T transpose la matrice pour obtenir une ligne unique de valeurs moyennes
    combined_data_df = pd.DataFrame(combined_data_numeric).T

    predicted_features = {}
    for feature in features:
        combined_data_scaled = scalers[feature].transform(combined_data_df.drop(columns=[feature]))
        predicted_features[feature] = models[feature].predict(combined_data_scaled)[0]

    # Créer un DataFrame avec les résultats
    results = pd.DataFrame({
        'HomeTeam': [home_team],
        'AwayTeam': [away_team],
        'FTHG': [predicted_features['FTHG']],
        'FTAG': [predicted_features['FTAG']],
        'FTR': [np.nan],  # La colonne FTR est initialisée avec NaN pour être prédite ensuite
        'HTGS': [predicted_features['HTGS']],
        'ATGS': [predicted_features['ATGS']],
        'HTGC': [predicted_features['HTGC']],
        'ATGC': [predicted_features['ATGC']],
        'HTP': [predicted_features['HTP']],
        'ATP': [predicted_features['ATP']],
        'MW': [predicted_features['MW']],
        'HTFormPts': [predicted_features['HTFormPts']],
        'ATFormPts': [predicted_features['ATFormPts']],
        'HTWinStreak3': [predicted_features['HTWinStreak3']],
        'HTWinStreak5': [predicted_features['HTWinStreak5']],
        'HTLossStreak3': [predicted_features['HTLossStreak3']],
        'HTLossStreak5': [predicted_features['HTLossStreak5']],
        'ATWinStreak3': [predicted_features['ATWinStreak3']],
        'ATWinStreak5': [predicted_features['ATWinStreak5']],
        'ATLossStreak3': [predicted_features['ATLossStreak3']],
        'ATLossStreak5': [predicted_features['ATLossStreak5']],
        'HTGD': [predicted_features['HTGD']],
        'ATGD': [predicted_features['ATGD']],
        'DiffPts': [predicted_features['DiffPts']],
        'DiffFormPts': [predicted_features['DiffFormPts']]
    })

    # Enregistrer les résultats dans un fichier CSV avec date et noms d'équipe
    current_date = datetime.now().strftime("%Y%m%d")

    # Spécifier le chemin où vous souhaitez enregistrer le fichier CSV
    output_path = 'Prédictions à prédire'
    file_name = f'predicted_{home_team}_{away_team}_{current_date}.csv'
    file_path = os.path.join(output_path, file_name)

    results.to_csv(file_path, index=False)
    print(f"Les prédictions du match sont stockées dans 'predicted_{home_team}_{away_team}_{current_date}.csv'")
    
else:
    print("Les données pour les équipes spécifiées ne sont pas disponibles dans le dataset.")

Les prédictions du match sont stockées dans 'predicted_Man City_Arsenal_20240627.csv'
